In [1]:
pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.6 MB/s eta 0:00:00


In [4]:
import sqlite3
import pandas as pd
from typing import List
from datetime import timedelta
from tqdm.notebook import tqdm
from xgboost import XGBRegressor
import ipywidgets as widgets
from IPython.display import display

In [ ]:
class EnergyPriceForecaster:
    """
    A class used to forecast electricity prices for the day-ahead market.
    """

    def __init__(self, database_path: str):
        self.database_path = database_path
        self.data = None

    def load_data(self, auctions: List[str] = None) -> pd.DataFrame:
        """
        Load the input dataset from the given SQLite database.
        """
        conn = sqlite3.connect(self.database_path)

        if auctions is None:
            auctions = ['DAM', 'IDA1', 'IDA2', 'IDA3']

        query = f"SELECT timestamp, price, auction FROM prices WHERE auction IN ({','.join('?' for _ in auctions)})"
        prices = pd.read_sql_query(query, conn, params=auctions, parse_dates=['timestamp'])

        features = pd.read_sql_query("SELECT * FROM features", conn, parse_dates=['timestamp'])

        data = pd.merge(prices, features, how='inner', on='timestamp')
        data.set_index('timestamp', inplace=True)

        conn.close()

        return data

    @staticmethod
    def prepare_features(data: pd.DataFrame) -> pd.DataFrame:
        """
        Pre-processes the input dataset and creates additional features.
        """
        data = data.dropna()
        return data

    def train_and_predict(self, train_end: pd.Timestamp, forecast_start: pd.Timestamp, forecast_end: pd.Timestamp) -> pd.DataFrame:
        """
        Trains the XGBoost model and predicts the energy prices.
        """
        prepared_data = self.prepare_features(self.data)
        train_data = prepared_data.loc[:train_end]
        test_data = prepared_data.loc[forecast_start:forecast_end]

        results_df = pd.DataFrame()

        for auction in self.data['auction'].unique():
            train_auction = train_data[train_data['auction'] == auction]
            test_auction = test_data[test_data['auction'] == auction]

            if train_auction.empty or test_auction.empty:
                continue

            X_train = train_auction.drop(columns=['price', 'auction'])
            y_train = train_auction['price']
            X_test = test_auction.drop(columns=['price', 'auction'])
            y_test = test_auction['price']

            model = XGBRegressor(n_estimators=1000, learning_rate=0.1)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)

            temp_df = pd.DataFrame({'timestamp': y_test.index,
                                    'auction': auction,
                                    'actual_price': y_test.values,
                                    'forecast_price': y_pred})
            results_df = pd.concat([results_df, temp_df], ignore_index=True)

        return results_df

    def forecast(self, start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
        """
        Forecasts the energy prices for a given range of dates.
        """
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')

        results_df = pd.DataFrame()

        for date in tqdm(date_range, desc="Forecasting"):
            train_end = date - timedelta(days=1)
            forecast_start = date
            forecast_end = date + timedelta(days=1)

            temp_df = self.train_and_predict(train_end, forecast_start, forecast_end)

            results_df = pd.concat([results_df, temp_df], ignore_index=True)

        return results_df

# Initialize the forecaster
database_path = 'your_database_path.sqlite'
forecaster = EnergyPriceForecaster(database_path)

# Create the widgets
date_picker = widgets.DatePicker(
    description='Delivery Date:',
    value=pd.Timestamp('2019-01-01')
)

auction_selector = widgets.SelectMultiple(
    options=['DAM', 'IDA1', 'IDA2', 'IDA3'],
    value=['DAM', 'IDA1', 'IDA2', 'IDA3'],
    description='Auctions:'
)

button = widgets.Button(
    description='Start Forecasting'
)

output = widgets.Output()

# Define what happens when the button is clicked
def on_button_clicked(_):
    with output:
        start_date = end_date = pd.Timestamp(date_picker.value)
        auctions = list(auction_selector.value)
        forecaster.data = forecaster.load_data(auctions)
        results_df = forecaster.forecast(start_date, end_date)
        print(results_df)

button.on_click(on_button_clicked)

# Display the widgets
display(date_picker, auction_selector, button, output)
